# ETL Project

Use Pandas scraping to help ...

In [103]:
import pandas as pd
import os
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import requests
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import getpass

In [114]:
#Kaggle
file_path = 'data/pb_winning_numbers_03-18-2017.csv'
pb = pd.read_csv(file_path)
pb_clean = pb[pb['DrawDate'] < '01/01/2010']
pb.dtypes

DrawDate     object
WB1           int64
WB2           int64
WB3           int64
WB4           int64
WB5           int64
PB            int64
PP          float64
dtype: object

In [116]:
pb['DrawDate']= pd.to_datetime(pb['DrawDate']) 

In [117]:
pb_new = pb.loc[pb['DrawDate'] >= '01/01/2010']
pb_new.count

<bound method DataFrame.count of       DrawDate  WB1  WB2  WB3  WB4  WB5  PB    PP
0   2017-03-15   30   16   41   48   53  16   3.0
1   2017-03-11   57   41   50   26    1  11   2.0
2   2017-03-08   42   23   59   46   33   4   2.0
3   2017-03-04    2   22   63   19   18  19   3.0
4   2017-03-01   16   10   52   40   55  17  10.0
5   2017-02-25    6   62   32   47   65  19   2.0
6   2017-02-22   52   10   61   28   13   2   2.0
7   2017-02-18    9    3   33    7   31  20   3.0
8   2017-02-15   33    5   42   38   28  19   2.0
9   2017-02-11   37   64   17    9    5   2   2.0
10  2017-02-08   49   20   42   14   66   5   2.0
11  2017-02-04   52   17   16   13    6  25   3.0
12  2017-02-01    9   60   64   43   57  10   2.0
13  2017-01-28   39   12   49   69   20  17   2.0
14  2017-01-25   18   62   66   68   28  22   2.0
15  2017-01-21   45   25   23   67   52   2   2.0
16  2017-01-18   41    9   58   40   53  12   2.0
17  2017-01-14   55   69   23   64   59  13   5.0
18  2017-01-11   

In [143]:
#TX Lottery
texas_url = "https://www.txlottery.org/export/sites/lottery/Games/Powerball/Winning_Numbers/index.html_1444349437.html"

texas_data = pd.read_html(texas_url)
texas_df = texas_data[0]

In [153]:
texas_df_new = texas_df.loc[(texas_df['Draw Date'] != 'First Drawing for new Powerball matrix was 10/07/2015')]
texas_df_new1 = texas_df_new.loc[(texas_df_new['Draw Date'] != 'First Drawing for new Powerball matrix was 10/07/2015.')]
texas_df_new2 = texas_df_new1.loc[(texas_df_new1['Draw Date'] != 'First Drawing for new $2 Powerball matrix was 1/18/2012.')]
texas_df_new3 = texas_df_new2.loc[(texas_df_new2['Draw Date'] >= '01/01/2010')]
texas_df = texas_df_new3.loc[(texas_df_new3['Draw Date'] <= '03/17/2017')]

In [154]:
texas_df.dtypes

Draw Date             object
Winning Numbers       object
Powerball            float64
Power Play           float64
Estimated Jackpot     object
Jackpot Winners       object
Jackpot Option        object
dtype: object

In [155]:
texas_df['Draw Date']= pd.to_datetime(texas_df['Draw Date']) 

/Users/amyclaman/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [156]:
texas_df.head()

,Draw Date,Winning Numbers,Powerball,Power Play,Estimated Jackpot,Jackpot Winners,Jackpot Option
14,2019-03-16,67 - 30 - 34 - 39 - 53,11.0,2.0,$495 Million,Roll,NaN
15,2019-03-13,69 - 36 - 45 - 47 - 18,14.0,3.0,$448 Million,Roll,NaN
16,2019-03-09,45 - 5 - 59 - 55 - 6,14.0,3.0,$414 Million,Roll,NaN
17,2019-03-06,21 - 35 - 10 - 6 - 46,23.0,2.0,$381 Million,Roll,NaN
18,2019-03-02,27 - 68 - 25 - 19 - 1,21.0,2.0,$348 Million,Roll,NaN


In [157]:

#texas_df_new2.rename(columns={'Draw Date':'DrawDate', 'Winning Numbers':'winning_numbers', 'Powerball':'powerball', 'Power Play':'power_play','Estimated Jackpot':'estimated_jackpot', 'Jackpot Winners':'jackpot_winner', 'Jackpot Option':'jackpot_option'}, inplace=True)
#texas_df = texas_df.drop(columns=['winning_numbers'])

clean_texas = texas_df[['Draw Date', 'Estimated Jackpot']]
clean_texas.head()


,Draw Date,Estimated Jackpot
14,2019-03-16,$495 Million
15,2019-03-13,$448 Million
16,2019-03-09,$414 Million
17,2019-03-06,$381 Million
18,2019-03-02,$348 Million


In [82]:
sales_file = "./data/powerball_sales.xlsx"
sales_df = pd.read_excel(sales_file)

sales_df.head()

,Date,Number of Tickets Sold,Jackpot (Millions)
0,2019-04-27,17204152,166.0
1,2019-04-24,15122600,150.0
2,2019-04-20,16422502,136.0
3,2019-04-17,14472208,118.0
4,2019-04-13,15663006,101.0


In [83]:
sales_df.columns

Index(['Date', 'Number of Tickets Sold', 'Jackpot (Millions)'], dtype='object')

In [84]:
sales_df['Number of Tickets Sold']= pd.to_numeric(sales_df['Number of Tickets Sold']) 

In [85]:
sales_df.columns = ['draw_date', 'tickets_sold', 'jackpot_millions']

In [86]:
sales_df.set_index('draw_date', inplace=True)

In [87]:
sales_df.dtypes

tickets_sold          int64
jackpot_millions    float64
dtype: object

In [101]:
from sqlalchemy import create_engine
engine = create_engine('sqlite://', echo=False)
conn = engine.connect()

In [104]:
from sqlalchemy import create_engine
import mysql.connector
SQLUSER = 'root'
SQLPASS = getpass.getpass()
SQLHOST = 'localhost'
SQLPORT = '3306'
engine = create_engine(f'mysql+mysqlconnector://{SQLUSER}:{SQLPASS}@{SQLHOST}:{SQLPORT}')
conn = engine.connect()

········


In [97]:
conn.execute('DROP DATABASE IF EXISTS lottery;')
conn.execute("CREATE DATABASE lottery")
lottery_database = engine.execute("USE lottery;")
sales_df.to_sql(name='sales', con=engine, if_exists='replace')


In [94]:
import sqlite3
#connect to the database
new_conn = sqlite3.connect('lottery')
c = new_conn.cursor()

c.executescript('''
    PRAGMA foreign_keys=off;

    BEGIN TRANSACTION;
    ALTER TABLE sales RENAME TO old_sales;

    /*create a new table with the same column names and types while
    defining a primary key for the desired column*/
    CREATE TABLE sales (draw_date DATETIME PRIMARY KEY NOT NULL,
                        tickets_sold INTEGER,
                        jackpot_millions FLOAT);

    INSERT INTO sales SELECT * FROM old_sales;

    DROP TABLE old_sales;
    COMMIT TRANSACTION;

    PRAGMA foreign_keys=on;''')

#close out the connection
c.close()
new_conn.close()

OperationalError: no such table: sales

In [99]:
sales_data = pd.read_sql('SELECT * FROM sales', engine)

In [100]:
sales_data

,draw_date,tickets_sold,jackpot_millions
0,2019-04-27,17204152,166.0
1,2019-04-24,15122600,150.0
2,2019-04-20,16422502,136.0
3,2019-04-17,14472208,118.0
4,2019-04-13,15663006,101.0
5,2019-04-10,13260953,84.0
6,2019-04-06,14525590,68.0
7,2019-04-03,13218913,54.0
8,2019-03-30,14970003,40.0
9,2019-03-27,134060942,750.0


In [98]:
engine.execute("SELECT * FROM sales").fetchall()


[(datetime.datetime(2019, 4, 27, 0, 0), 17204152, 166.0),
 (datetime.datetime(2019, 4, 24, 0, 0), 15122600, 150.0),
 (datetime.datetime(2019, 4, 20, 0, 0), 16422502, 136.0),
 (datetime.datetime(2019, 4, 17, 0, 0), 14472208, 118.0),
 (datetime.datetime(2019, 4, 13, 0, 0), 15663006, 101.0),
 (datetime.datetime(2019, 4, 10, 0, 0), 13260953, 84.0),
 (datetime.datetime(2019, 4, 6, 0, 0), 14525590, 68.0),
 (datetime.datetime(2019, 4, 3, 0, 0), 13218913, 54.0),
 (datetime.datetime(2019, 3, 30, 0, 0), 14970003, 40.0),
 (datetime.datetime(2019, 3, 27, 0, 0), 134060942, 750.0),
 (datetime.datetime(2019, 3, 23, 0, 0), 90298137, 625.0),
 (datetime.datetime(2019, 3, 20, 0, 0), 71029841, 550.0),
 (datetime.datetime(2019, 3, 16, 0, 0), 46498176, 495.0),
 (datetime.datetime(2019, 3, 13, 0, 0), 37293378, 448.0),
 (datetime.datetime(2019, 3, 9, 0, 0), 34396266, 414.0),
 (datetime.datetime(2019, 3, 6, 0, 0), 26004175, 381.0),
 (datetime.datetime(2019, 3, 2, 0, 0), 26259031, 348.0),
 (datetime.datetime(20

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

class Sales(Base):
  __tablename__ = "sales"

  draw_date = Column(datetime, primary_key=True)
  tickets_sold = Column(Integer)
  jackpot_millions = Column(Float)
    
    def __repr__(self):
        return "(Draw Date ='%s', Number of Tickets Sold ='%s', Jackpot (Millions) ='%s')" % (self.draw_date, self.tickets_sold, self.jackpot_millions)

  